<a href="https://colab.research.google.com/github/WURXPP/Hello-world/blob/master/TestDWPointsRunner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CONFIG

# New Section

In [ ]:
# POINTS_URL/FILENAME: location in GCS of a csv containing a "lat" and "lon" columns
# - to run accuracy analysis you also need to have reference-label column whose name is given by REFERNCE_COL below
POINTS_FILENAME='dev_dw_sample_pts-500.csv'
POINTS_URL=f'https://storage.googleapis.com/dynamic-world-public/dw-exports/point_data/{POINTS_FILENAME}'


# DESTINATION_GCS_FOLDER_URI: location of folder in GCS to save the output files
# - if you run the accuracy analysis "accuracy/" and "confusion/" subfolders will be created
DESTINATION_GCS_FOLDER_URI='gs://dynamic-world-public/dwpoints'


# REFERNCE_COL: the name of the col in the POINTS file that has the reference (truth) label
REFERENCE_COL='label'

## INSTALL (DWPOINTS, EE-API, GCSFS, (update) PANDAS)

In [ ]:
!rm -rf dwpoints 
!git clone https://github.com/wri/dwpoints.git
%cd dwpoints
!pip install -e .

In [ ]:
!pip install earthengine-api

In [ ]:
!pip install gcsfs

In [ ]:
!pip install pandas -U

## IMPORTS

In [ ]:
import re
import glob
import pandas as pd
from google.colab import auth
import ee
import dwpoints

## AUTHENTICATE COLAB AND GEEE

In [ ]:
auth.authenticate_user()

In [ ]:
ee.Authenticate()

## CREATE WORKING DIRECTORY

In [ ]:
%cd ..
%mkdir DWPOINTS_OUTPUT
%cd DWPOINTS_OUTPUT

/content
/content/DWPOINTS_OUTPUT


## RUN DWPOINTS AND SAVE TO GCS

In [ ]:
!dwpoints annual {POINTS_URL}

Upload the points as csv
Navigate to the csv file in your local drive and upload. 

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving testSubset.csv to testSubset.csv


In [ ]:
!dwpoints annual testSubset.csv --2021

In [ ]:
!gsutil cp dwpoints.2022.testSubset.csv {DESTINATION_GCS_FOLDER_URI}

## VISUAL INSPECTION

In [ ]:
dfLoc=pd.read_csv('testSubset.csv')
dfDW=pd.read_csv('dwpoints.2022.testSubset.csv')
print('NB_ROWS:',dfLoc.shape[0]) #nrow extract
print('NB_ROWS:',dfDW.shape[0]) #nrow extract

NB_ROWS: 1400
NB_ROWS: 1400


In [ ]:
print(dfLoc.head(n=10).to_string(index=False))
print(df.head(n=10).to_string(index=False))
print(df.tail(n=10).to_string(index=False))

      lon       lat  subid  sample_id_20  orig_sampleid_20  validation_id_20  location_id_20
31.234730 31.255347      1       1020542           1020542           1726062         1953547
31.235570 31.255356      2       1020542           1020542           1726062         1953547
31.235147 31.255532      3       1020542           1020542           1726062         1953547
31.234736 31.254986      4       1020542           1020542           1726062         1953547
31.234941 31.255259      5       1020542           1020542           1726062         1953547
31.234940 31.255349      6       1020542           1020542           1726062         1953547
31.234838 31.255168      7       1020542           1020542           1726062         1953547
31.234935 31.255710      8       1020542           1020542           1726062         1953547
31.234727 31.255527      9       1020542           1020542           1726062         1953547
31.235575 31.254996     10       1020542           1020542           1

Download the extraxted values

In [ ]:
from google.colab import files
df.to_csv('dwpoints.2022.testSubset.csv') 
files.download('dwpoints.2022.testSubset.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# ACCURACY ANALYSIS: Skip if uninterested
#to skip...

In [ ]:
if not REFERENCE_COL:
  raise ValueError('REFERENCE_COL required to run analysis')


## ACCURACY 

In [ ]:
!dwpoints accuracy dwpoints.2022.{POINTS_FILENAME} {REFERENCE_COL}

[INFO] DW_POINTS: generating aggrement assement
----------------------------------------------------------------------------------------------------
{'dest': 'acc.dwpoints.2022.dev_dw_sample_pts-500.csv',
 'nb_points': 4500,
 'squash_columns': ['dw_median_label', 'dw_median_cr', 'dw_median_sr_cr_br'],
 'src': 'dwpoints.2022.dev_dw_sample_pts-500.csv'}
[INFO] DW_POINTS: [[2023.02.15] 19:17:24] ...
[INFO] DW_POINTS: [[2023.02.15] 19:17:25] complete (0:00:00.135288)


In [ ]:
!gsutil cp acc.dwpoints.2022.{POINTS_FILENAME} {DESTINATION_GCS_FOLDER_URI}/accuracy/

Copying file://acc.dwpoints.2022.dev_dw_sample_pts-500.csv [Content-Type=text/csv]...
/ [1 files][  591.0 B/  591.0 B]                                                
Operation completed over 1 objects/591.0 B.                                      


In [ ]:
acc_df=pd.read_csv(f'{DESTINATION_GCS_FOLDER_URI}/accuracy/acc.dwpoints.2022.{POINTS_FILENAME}')
acc_df.set_index(REFERENCE_COL)

,dw_median_label_acc,dw_median_cr_acc,dw_median_sr_cr_br_acc,dw_median_label_count,dw_median_cr_count,dw_median_sr_cr_br_count,total
label,,,,,,,
0.0,0.984,0.984,0.984000,492.0,492.0,492.0,500.0
1.0,0.986,0.986,0.986000,493.0,493.0,493.0,500.0
2.0,0.952,0.952,0.952000,476.0,476.0,476.0,500.0
3.0,0.824,0.824,0.824000,412.0,412.0,412.0,500.0
4.0,0.946,0.946,0.946000,473.0,473.0,473.0,500.0
5.0,0.912,0.912,0.912000,456.0,456.0,456.0,500.0
6.0,0.988,0.988,0.988000,494.0,494.0,494.0,500.0
7.0,0.776,0.776,0.778000,388.0,388.0,389.0,500.0
8.0,0.732,0.732,0.000000,366.0,366.0,0.0,500.0


## CONFUSION MATRICES

In [ ]:
!dwpoints confusion dwpoints.2022.{POINTS_FILENAME} {REFERENCE_COL}
!dwpoints confusion dwpoints.2022.{POINTS_FILENAME} {REFERENCE_COL} --normalize True

[INFO] DW_POINTS: generating confusion matrices
----------------------------------------------------------------------------------------------------
{'dest': 'cm.<squash>.dwpoints.2022.dev_dw_sample_pts-500.csv',
 'nb_points': 4500,
 'squash_columns': ['dw_median_label', 'dw_median_cr', 'dw_median_sr_cr_br'],
 'src': 'dwpoints.2022.dev_dw_sample_pts-500.csv'}
[INFO] DW_POINTS: [[2023.02.15] 19:18:28] ...
- cm.dw_median_label.dwpoints.2022.dev_dw_sample_pts-500.csv
[INFO] DW_POINTS: [[2023.02.15] 19:18:28] dw_median_label (0:00:00.010427)
- cm.dw_median_cr.dwpoints.2022.dev_dw_sample_pts-500.csv
[INFO] DW_POINTS: [[2023.02.15] 19:18:28] dw_median_cr (0:00:00.018706)
- cm.dw_median_sr_cr_br.dwpoints.2022.dev_dw_sample_pts-500.csv
[INFO] DW_POINTS: [[2023.02.15] 19:18:28] dw_median_sr_cr_br (0:00:00.026902)
[INFO] DW_POINTS: [[2023.02.15] 19:18:28] complete (0:00:00.026935)
[INFO] DW_POINTS: generating confusion matrices
--------------------------------------------------------------------

In [ ]:
%ls

acc.dwpoints.2022.dev_dw_sample_pts-500.csv
cm.dw_median_cr.dwpoints.2022.dev_dw_sample_pts-500.csv
cm.dw_median_label.dwpoints.2022.dev_dw_sample_pts-500.csv
cm.dw_median_sr_cr_br.dwpoints.2022.dev_dw_sample_pts-500.csv
cm-norm.dw_median_cr.dwpoints.2022.dev_dw_sample_pts-500.csv
cm-norm.dw_median_label.dwpoints.2022.dev_dw_sample_pts-500.csv
cm-norm.dw_median_sr_cr_br.dwpoints.2022.dev_dw_sample_pts-500.csv
dwpoints.2022.dev_dw_sample_pts-500.csv


In [ ]:
CM_FILES=glob.glob('cm.*.2022.*')

In [ ]:
CM_NORM_FILES=glob.glob('cm-norm.*')

In [ ]:
def display_cm(path):
  print()
  print('-'*100)
  print(re.sub('\.csv','',path).upper())
  print('-'*100)
  print()
  _df=pd.read_csv(path)
  display(_df.set_index('label'))


def save_cm(path):
  !gsutil cp {path} {DESTINATION_GCS_FOLDER_URI}/confusion/

In [ ]:
print('\n'*2)
print('CONFUSION MATRICES [pixel-count]')
for path in CM_FILES:
  display_cm(path)
  save_cm(path)




CONFUSION MATRICES [pixel-count]

----------------------------------------------------------------------------------------------------
CM.DW_MEDIAN_SR_CR_BR.DWPOINTS.2022.DEV_DW_SAMPLE_PTS-500
----------------------------------------------------------------------------------------------------



,DW_0,DW_1,DW_2,DW_3,DW_4,DW_5,DW_6,DW_7,DW_8,total
label,,,,,,,,,,
0,492,6,1,0,0,1,0,0,0,500
1,0,493,2,0,1,3,1,0,0,500
2,0,4,476,0,12,7,1,0,0,500
3,13,23,7,412,2,39,0,4,0,500
4,0,4,16,0,473,7,0,0,0,500
5,0,28,11,0,2,456,1,2,0,500
6,0,5,1,0,0,0,494,0,0,500
7,0,1,2,0,4,103,1,389,0,500
8,38,73,6,0,2,101,104,176,0,500


Copying file://cm.dw_median_sr_cr_br.dwpoints.2022.dev_dw_sample_pts-500.csv [Content-Type=text/csv]...
-
Operation completed over 1 objects/351.0 B.                                      

----------------------------------------------------------------------------------------------------
CM.DW_MEDIAN_CR.DWPOINTS.2022.DEV_DW_SAMPLE_PTS-500
----------------------------------------------------------------------------------------------------



,DW_0,DW_1,DW_2,DW_3,DW_4,DW_5,DW_6,DW_7,DW_8,total
label,,,,,,,,,,
0,492,6,1,0,0,1,0,0,0,500
1,0,493,2,0,1,3,1,0,0,500
2,0,4,476,0,12,7,1,0,0,500
3,13,23,7,412,2,39,0,4,0,500
4,0,4,16,0,473,7,0,0,0,500
5,0,28,11,0,2,456,1,2,0,500
6,0,5,1,0,0,0,494,0,0,500
7,0,1,2,0,4,103,1,388,1,500
8,2,54,3,0,0,41,16,18,366,500


Copying file://cm.dw_median_cr.dwpoints.2022.dev_dw_sample_pts-500.csv [Content-Type=text/csv]...
/ [1 files][  351.0 B/  351.0 B]                                                
Operation completed over 1 objects/351.0 B.                                      

----------------------------------------------------------------------------------------------------
CM.DW_MEDIAN_LABEL.DWPOINTS.2022.DEV_DW_SAMPLE_PTS-500
----------------------------------------------------------------------------------------------------



,DW_0,DW_1,DW_2,DW_3,DW_4,DW_5,DW_6,DW_7,DW_8,total
label,,,,,,,,,,
0,492,6,1,0,0,1,0,0,0,500
1,0,493,2,0,1,3,1,0,0,500
2,0,4,476,0,12,7,1,0,0,500
3,13,23,7,412,2,39,0,4,0,500
4,0,4,16,0,473,7,0,0,0,500
5,0,28,11,0,2,456,1,2,0,500
6,0,5,1,0,0,0,494,0,0,500
7,0,1,2,0,4,103,1,388,1,500
8,2,54,3,0,0,41,16,18,366,500


Copying file://cm.dw_median_label.dwpoints.2022.dev_dw_sample_pts-500.csv [Content-Type=text/csv]...
-
Operation completed over 1 objects/351.0 B.                                      


In [ ]:
print('\n'*2)
print('CONFUSION MATRICES [normalize]')
for path in CM_NORM_FILES:
  display_cm(path)




CONFUSION MATRICES [normalize]

----------------------------------------------------------------------------------------------------
CM-NORM.DW_MEDIAN_LABEL.DWPOINTS.2022.DEV_DW_SAMPLE_PTS-500
----------------------------------------------------------------------------------------------------



,DW_0,DW_1,DW_2,DW_3,DW_4,DW_5,DW_6,DW_7,DW_8,total
label,,,,,,,,,,
0,0.970414,0.009709,0.001927,0.0,0.000000,0.001522,0.000000,0.000000,0.000000,500
1,0.000000,0.797735,0.003854,0.0,0.002024,0.004566,0.001946,0.000000,0.000000,500
2,0.000000,0.006472,0.917148,0.0,0.024291,0.010654,0.001946,0.000000,0.000000,500
3,0.025641,0.037217,0.013487,1.0,0.004049,0.059361,0.000000,0.009709,0.000000,500
4,0.000000,0.006472,0.030829,0.0,0.957490,0.010654,0.000000,0.000000,0.000000,500
5,0.000000,0.045307,0.021195,0.0,0.004049,0.694064,0.001946,0.004854,0.000000,500
6,0.000000,0.008091,0.001927,0.0,0.000000,0.000000,0.961089,0.000000,0.000000,500
7,0.000000,0.001618,0.003854,0.0,0.008097,0.156773,0.001946,0.941748,0.002725,500
8,0.003945,0.087379,0.005780,0.0,0.000000,0.062405,0.031128,0.043689,0.997275,500



----------------------------------------------------------------------------------------------------
CM-NORM.DW_MEDIAN_CR.DWPOINTS.2022.DEV_DW_SAMPLE_PTS-500
----------------------------------------------------------------------------------------------------



,DW_0,DW_1,DW_2,DW_3,DW_4,DW_5,DW_6,DW_7,DW_8,total
label,,,,,,,,,,
0,0.970414,0.009709,0.001927,0.0,0.000000,0.001522,0.000000,0.000000,0.000000,500
1,0.000000,0.797735,0.003854,0.0,0.002024,0.004566,0.001946,0.000000,0.000000,500
2,0.000000,0.006472,0.917148,0.0,0.024291,0.010654,0.001946,0.000000,0.000000,500
3,0.025641,0.037217,0.013487,1.0,0.004049,0.059361,0.000000,0.009709,0.000000,500
4,0.000000,0.006472,0.030829,0.0,0.957490,0.010654,0.000000,0.000000,0.000000,500
5,0.000000,0.045307,0.021195,0.0,0.004049,0.694064,0.001946,0.004854,0.000000,500
6,0.000000,0.008091,0.001927,0.0,0.000000,0.000000,0.961089,0.000000,0.000000,500
7,0.000000,0.001618,0.003854,0.0,0.008097,0.156773,0.001946,0.941748,0.002725,500
8,0.003945,0.087379,0.005780,0.0,0.000000,0.062405,0.031128,0.043689,0.997275,500



----------------------------------------------------------------------------------------------------
CM-NORM.DW_MEDIAN_SR_CR_BR.DWPOINTS.2022.DEV_DW_SAMPLE_PTS-500
----------------------------------------------------------------------------------------------------



,DW_0,DW_1,DW_2,DW_3,DW_4,DW_5,DW_6,DW_7,DW_8,total
label,,,,,,,,,,
0,0.906077,0.009419,0.001916,0.0,0.000000,0.001395,0.000000,0.000000,0.0,500
1,0.000000,0.773940,0.003831,0.0,0.002016,0.004184,0.001661,0.000000,0.0,500
2,0.000000,0.006279,0.911877,0.0,0.024194,0.009763,0.001661,0.000000,0.0,500
3,0.023941,0.036107,0.013410,1.0,0.004032,0.054393,0.000000,0.007005,0.0,500
4,0.000000,0.006279,0.030651,0.0,0.953629,0.009763,0.000000,0.000000,0.0,500
5,0.000000,0.043956,0.021073,0.0,0.004032,0.635983,0.001661,0.003503,0.0,500
6,0.000000,0.007849,0.001916,0.0,0.000000,0.000000,0.820598,0.000000,0.0,500
7,0.000000,0.001570,0.003831,0.0,0.008065,0.143654,0.001661,0.681261,0.0,500
8,0.069982,0.114600,0.011494,0.0,0.004032,0.140865,0.172757,0.308231,0.0,500
